# Recurrent Neural Networks : Long Short Term Memory networks (LSTM)

We have already covered feed-forward neural networks during the computer vision and the recommended system module. For natural language processing, one type of popular deep learning architecture is called Reccurent Neural Networks (RNNs). RNNs differ from feed-forward networks in the sense that some of their inner layers are recursively updated while iterating over the sequence of words given in input. We are going to use one specific RNN architecture called Long-Short Term Memory networks (LSTMs), which have been especially successful in various NLP tasks, including automatic translation, question answering, ... and text classification, our case study in this module.

Learn more about how RNNs and LSTMs encode texts as vectors first:

https://medium.com/towards-data-science/illustrated-guide-to-recurrent-neural-networks-79e5eb8049c9

If you want to understand in depth how one LSTM cell is working, you can go through these two articles:

https://medium.com/towards-data-science/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21

http://colah.github.io/posts/2015-08-Understanding-LSTMs/


## Load Data

In [ ]:
outdir = '../data/imdb_clean/'

In [ ]:
import pandas as pd
import numpy as np
train_deep_clean = pd.read_csv(outdir + 'train.csv')
valid_deep_clean = pd.read_csv(outdir + 'valid.csv')
test_deep_clean = pd.read_csv(outdir + 'test.csv')

## Implementing LSTMs with Keras

In [ ]:
#Some lines that allow for faster training with this version of tensorflow for these models
import tensorflow as tf
print(tf.__version__)

physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)

In [ ]:
import keras
from keras.layers import TextVectorization
from keras.models import Sequential
from keras.layers import *

We will use Keras to implement an LSTM network. First we need to encode all the reviews as a list of indexes, where each word is replaced by its embedding index using keras `TextVectorization` object. To make all training reviews the same size, the `TextVectorization` will add a special token < pad > (encoded as 0) as many time as necessary to each text to match the length of the longest text. This token will be ignored by the LSTM layer.

In [ ]:
max_vocab_size = 10000 

vectorizer = TextVectorization(max_tokens=max_vocab_size, output_mode='int',split='whitespace')
vectorizer.adapt(pd.concat([train_deep_clean,valid_deep_clean]).review)

# This encodes our sentence as a sequence of integer
# each integer being the index of each word in the vocabulary
# Zeros are added at the end so that each text has the same length
train_seqs = vectorizer(train_deep_clean.review)
valid_seqs = vectorizer(valid_deep_clean.review)
test_seqs = vectorizer(test_deep_clean.review)

max_seq_length = train_seqs.shape[1]

#However due to a bug in Keras 3, we need to put the zeros at the beginning, not at the end:
def repad_left(x, maxlen):       
    rows = []
    x_arr = np.array(x)
    x_arr = np.concatenate((x_arr, np.zeros((x_arr.shape[0],1))), axis=1)
    for row in x_arr:
        rows.append(row[:row.argmin()])
    ledt_padded_x = keras.utils.pad_sequences(rows, maxlen=maxlen)
    return ledt_padded_x
        

X_train = repad_left(train_seqs, max_seq_length)
X_valid = repad_left(valid_seqs, max_seq_length)
X_test = repad_left(test_seqs, max_seq_length)

#Finally we encode the ys :
y_train = pd.get_dummies(train_deep_clean.sentiment).values[:,1]
y_valid = pd.get_dummies(valid_deep_clean.sentiment).values[:,1]
y_test = pd.get_dummies(test_deep_clean.sentiment).values[:,1]


Now fill the following function to implement a simple LSTM model : one embedding layer, one LSTM layer, and a final dense layer that yields a single score with a sigmoid activation function. Use Keras' Sequential API.

In [ ]:
def get_lstm_model(vocab_size, embedding_dim, seq_length, lstm_out_dim):
    #TOFILL

    
    #TOKEEP
    model.compile(loss = 'binary_crossentropy', optimizer='SGD',metrics = ['accuracy'])
    return model

In [ ]:
embedding_dim = 100
lstm_out_dim = 200  #Bigger than embedding dim, as it combines all the words of each review

model = get_lstm_model(max_vocab_size, embedding_dim, max_seq_length, lstm_out_dim)
print(model.summary())

In [ ]:
batch_size = 64
max_epochs = 2
history = model.fit(X_train, y_train, epochs=max_epochs, batch_size=batch_size, 
                    verbose=1, validation_data = (X_valid, y_valid))

In [ ]:
test_acc = model.evaluate(X_test, y_test, verbose=0) 
print("Test accuracy: %.2f%%" % (test_acc[1]*100))

Pretty low accuracy isn't it ? Actually it is very easy to incorrectly train a deep neural net. Change the optimizer with "adam" instead of "SGD", add a dropout layer after the LSTM layer for regularization, and use early stopping :

In [ ]:
#dropout, early stopping, adam
def get_lstm_model_2(vocab_size, embedding_dim, seq_length, lstm_out_dim, dropout_rate):
    #TOFILL

    

In [ ]:
embedding_dim = 100
lstm_out_dim = 200
dropout_rate = 0.2

model = get_lstm_model_2(max_vocab_size, embedding_dim, max_seq_length, lstm_out_dim, dropout_rate)
print(model.summary())

In [ ]:
#TOFILL
early_stopping = 

In [ ]:
batch_size = 32
max_epochs = 5
history = model.fit(X_train, y_train, epochs=max_epochs, batch_size=batch_size, 
                    verbose=1, validation_data = (X_valid, y_valid), callbacks=[early_stopping])


In [ ]:
test_acc = model.evaluate(X_test, y_test, verbose=0)
print("Test accuracy: %.2f%%" % (test_acc[1]*100))

Much better. If we'd run for a longer time, we'd get a bit better results from our classic methods, but that's still quite slow for little improvement. We could also grid search for all hyper parameters (embedding and layer sizes, dropout rate, ...), but that's not the goal today, remember however that grid-search is standard when optimizing a model predictive performances.

## Predict sentiment for arbitrary sentences

Now you can try predict the sentiment of any kind of sentence in english, try your own. You first need to encode each review as a sequence of indexes (called tokens in keras), to pad these sequances, and finally predict the score with your trained model:

In [ ]:
good = "i really liked the movie and had fun"
bad = "worst movie on the planet , so boring"
for review in [good,bad]:
    
    #TOFILL

## Initialize embeddings with pre-trained word embeddings

The training of LSTMs is a bit heavy, one way to speed this up is to re-use pre-trained word embeddings. Many such embeddings are available on the net. Read this to understand how are produced word embeddings and why they encode information that helps with all NLP tasks:

http://jalammar.github.io/illustrated-word2vec/

We are going to use GloVe embeddings, download and load the embeddings produced from 6 billions documents from : https://nlp.stanford.edu/projects/glove/

In [ ]:
embeddings_index = {}
f = open('../data/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Loaded %s word vectors.' % len(embeddings_index))

In [ ]:
word_index = {w : i for i,w in enumerate(vectorizer.get_vocabulary()) }

In [ ]:
print('%s unique words in vocabulary' % len(word_index))

Given our word index, search for each of our 10000 most frequents words if they exist in the pretrained GloVe embeddings and assign them to their corresponding row index in the embedding matrix. If they don't exist in the GloVe embeddings, assign a random vector :

In [ ]:
embedding_dim = 100

# Allocate the embeddings matrix
embedding_matrix = np.zeros((max_vocab_size, embedding_dim))


for word, i in word_index.items():
    #TOFILL


Now change your LSTM model so that the embedding layer is initialized with the pretrained embeddings :

In [ ]:
def get_lstm_model_pretrained_embs(vocab_size, embedding_dim, seq_length, 
                                   lstm_out_dim, dropout_rate, embedding_matrix):
    #TOFILL
    

In [ ]:
embedding_dim = 100
lstm_out_dim = 200
dropout_rate = 0.2

model = get_lstm_model_pretrained_embs(max_vocab_size, embedding_dim, max_seq_length, 
                                       lstm_out_dim, dropout_rate, embedding_matrix)
print(model.summary())

In [ ]:
batch_size = 64
max_epochs = 5
history = model.fit(X_train, y_train, epochs=max_epochs, batch_size=batch_size, 
                    verbose=1, validation_data = (X_valid, y_valid), callbacks=[early_stopping])


In [ ]:
test_acc = model.evaluate(X_test, y_test, verbose=0) 
print("Test accuracy: %.2f%%" % (test_acc[1]*100))

We can see that the validation accuracy indeed progressed faster during the first epochs. For this small dataset it is not an issue, but it can save hours of training on bigger ones. It also reached a higher accuracy, which is not always the case, especially on bigger datasets.

For more speed-up, at the expense of accuracy, let's fix the embeddings so that they are not trainable parameters of the model, meaning they won't be updated during training :

In [ ]:
def get_lstm_model_pretrained_embs(vocab_size, embedding_dim, seq_length, 
                                   lstm_out_dim, dropout_rate, embedding_matrix,
                                    trainable_embeddings):
    #TOFILL
    

In [ ]:
embedding_dim = 100
lstm_out_dim = 200
dropout_rate = 0.2
trainable_embeddings = False

model = get_lstm_model_pretrained_embs(max_vocab_size, embedding_dim, max_seq_length, 
                                       lstm_out_dim, dropout_rate, embedding_matrix, trainable_embeddings)
print(model.summary())

Notice the change in the number of trainable parameters in the summary.

In [ ]:
batch_size = 64
max_epochs = 5
history = model.fit(X_train, y_train, epochs=max_epochs, batch_size=batch_size, 
                    verbose=1, validation_data = (X_valid, y_valid), callbacks=[early_stopping])


In [ ]:
test_acc = model.evaluate(X_test, y_test, verbose=0) 
print("Test accuracy: %.2f%%" % (test_acc[1]*100))

By fixing the word embeddings, the training time shrunk a bit, but the validation accuracy is progressing more slowly and reaching a limit. Depending on the network architecture, the trade-off can be interesting, here not so much, just know this is a possibility.

# Going further

## Bidirectional and stacked LSTMs

LSTMs parse the text from left to right, but doing it also from right to left and concatening the two output vectors improved the results. These are called bidirectional LSTMs. It is also possible to stack multiple LSTM layers.

This image is a good illustration of how these two variants work:

https://www.researchgate.net/figure/Illustrations-for-basic-LSTMs-and-the-three-layer-stacked-LSTM-model-for-the-sequential_fig3_313115860


First modify your network to make a bidirectional LSTM :

In [ ]:
def get_bilstm_model_pretrained_embs(vocab_size, embedding_dim, seq_length, 
                                     lstm_out_dim, dropout_rate, embedding_matrix,
                                    trainable_embeddings):
    #TOFILL
    

In [ ]:
embedding_dim = 100
lstm_out_dim = 200
dropout_rate = 0.2

model = get_bilstm_model_pretrained_embs(max_vocab_size, embedding_dim, max_seq_length, 
                                       lstm_out_dim, dropout_rate, embedding_matrix, True)

print(model.summary())

In [ ]:
batch_size = 32
max_epochs = 5
history = model.fit(X_train, y_train, epochs=max_epochs, batch_size=batch_size, 
                    verbose=1, validation_data = (X_valid, y_valid), callbacks=[early_stopping])


In [ ]:
test_acc = model.evaluate(X_test, y_test, verbose=0) 
print("Test accuracy: %.2f%%" % (test_acc[1]*100))

Now try stacking multiple bidirectional LSTM layers, where the number of layers `n_layers` is a parameter of the function building the model :

In [ ]:
def get_multilayer_bilstm_model_pretrained_embs(vocab_size, embedding_dim, seq_length, 
                                                lstm_out_dim, dropout_rate, embedding_matrix,
                                                trainable_embeddings, n_layers):
    #TOFILL

In [ ]:
embedding_dim = 100
lstm_out_dim = 100
dropout_rate = 0.2
n_layers = 2

model = get_multilayer_bilstm_model_pretrained_embs(max_vocab_size, embedding_dim, max_seq_length, 
                                       lstm_out_dim, dropout_rate, embedding_matrix, True, n_layers)

print(model.summary())

In [ ]:
batch_size = 32
max_epochs = 5
history = model.fit(X_train, y_train, epochs=max_epochs, batch_size=batch_size, 
                    verbose=1, validation_data = (X_valid, y_valid), callbacks=[early_stopping])


In [ ]:
test_acc = model.evaluate(X_test, y_test, verbose=0) 
print("Test accuracy: %.2f%%" % (test_acc[1]*100))

As you can see, the max accuracy reached is not much better than our TF-IDF model. This happens because the full word order is actually not so important for Sentiment Analysis. For this task, Convolutional Neural Networks can attain comparable performances faster, as they have simpler architectures. But that's not true for other task such as translation, question answering, ... (which are tasks that are a bit too long to train to be included in this course, hence the choice of sentiment an analysis to practice RNNs).

Let's do it with a convolutional model by using 1D convolution with a kernel size of 3 over the word embeddings (this means that it will convolve the embeddings of the consecutive words 3 by 3), followed by a 1D max pooling and a dense ReLU layer before the final sigmoid :

In [ ]:
def get_conv_model_pretrained_embs(vocab_size, embedding_dim, seq_length, 
                                                filters, dropout_rate, embedding_matrix,
                                                trainable_embeddings):
    #TOFILL

In [ ]:
embedding_dim = 100
filters = 100
dropout_rate = 0.2

model = get_conv_model_pretrained_embs(max_vocab_size, embedding_dim, max_seq_length, 
                                       filters, dropout_rate, None, True)

print(model.summary())

In [ ]:
batch_size = 64
max_epochs = 5
history = model.fit(X_train, y_train, epochs=max_epochs, batch_size=batch_size, 
                    verbose=1, validation_data = (X_valid, y_valid), callbacks=[early_stopping])


In [ ]:
test_acc = model.evaluate(X_test, y_test, verbose=0) 
print("Test accuracy: %.2f%%" % (test_acc[1]*100))

# Going even further

The following parts are meant to be resources to explore if you are interested in the advanced concept of attention in deep nets. There are explanation links, as well as links with code for each of them, but don't feel obliged to implement all of them, these are meant to help understanding each of these concepts.

## Attention

Attention is a mechanism that changes the output of an LSTM : instead of outputting the final hidden state vector $h_n$ where $n$ is the length of the encoded text, attention plugs on top of a LSTM and returns a combination of all the hidden state vectors at each word position $\ \sum_{t=1}^n \alpha_t h_t$ (where $\alpha_t \in (0,1))$, and thus allows to pay a different attention to each part of the text, hence the name. 

It has been originally proposed for sequence to sequence models, like translation models, where there is a different attention combination computed for each translated output word. It is thus less useful for text classification, but it can be adapted, by computing a single output combination of all the hidden states, as explained in Section 3.3 of the following article :

https://www.aclweb.org/anthology/P16-2034.pdf

Here is a link about how to apply attention for text classification with Keras:

https://www.kaggle.com/yshubham/simple-lstm-for-text-classification-with-attention


You can also read the following link to understand how attention works in sequence to sequence models, which are nothing more than a reversed LSTM (the decoder) on top of a first LSTM (the encoder), in this case for translation where it helps aligning words in two different languages :

https://medium.com/towards-data-science/day-1-2-attention-seq2seq-models-65df3f49e263

## Transformer architecture for text classification

State of the art models in NLP are not RNNs anymore, but Transformers. Transformers do not read text sequentially like RNNs, the core concept of Transformers is self-attention, an attention mechanism that combine separately each word embedding with the other word embeddings of the text. There are multiple such attention mechanisms called "attention heads" in a layer, and multiple such layers are stacked.

Read this article to understand the self-attention layer:

https://medium.com/towards-data-science/illustrated-self-attention-2d627e33b20a

This article explains very well the Transformer for sequence to sequence models (again remember that a text classification model is just the encoder part of a sequence to sequence model) :

http://jalammar.github.io/illustrated-transformer/

Keras code to do text classification with a Transformer :

https://keras.io/examples/nlp/text_classification_with_transformer/



## BERT

Current state-of-the-art performance for text classification are achieved by doing transfer learning from the BERT model. The BERT model combines different techniques including the Transformer to pretrain in an unsupervised fashion on plain text. The last layers of BERT provide a high-level contextual representation of english sentences, and can then be reused in any NLP deep model. The website HuggingFace ( https://huggingface.co/ ) hosts many pretrained deep learning models that can be reused and fine-tuned for your application.

The BERT model : http://jalammar.github.io/illustrated-bert/

Reusing a pretrained BERT model from Keras-Hub for text classification : https://keras.io/keras_hub/api/models/bert/bert_text_classifier/#berttextclassifier-class

## BERT in pytorch with skorch

If you want to familizarize yourself with pytorch, one easy way when one is used to scikit-learn and keras is to use the `skorch` library. `skorch` implements a high level interface to pytorch with the usual `fit` and `predict` functions : https://skorch.readthedocs.io/en/stable/user/quickstart.html

You can first try to reimplement a simple LSTM with pytorch. Then you can try to redo the prediction with a BERT model from huggingface : https://nbviewer.org/github/skorch-dev/skorch/blob/master/notebooks/Hugging_Face_Finetuning.ipynb

## Querying OpenAI (ChatGPT, GPT, ...) models 

The largest language models available nowadays like the latest versions of GPT and ChatGPT do not fit on a typical computer, but we can query them through OpenAI API to use them. However this is not free :
https://openai.com/pricing

If you feel like you want to spend some money for this, we can try two different ways. In both cases we can use the `langchain` package to interact with the openAI API through python code. 

The first option is to get embeddings for each text from the Ada2 model, and then build a classifier from the embeddings as features :
https://shishirsingh66g.medium.com/langchain-applications-part-3-embedding-models-75e6a0d01545

Another one is simply to ask ChatGPT whether it thinks each review of the test set is positive or negative, and compute its accuracy. Look at the `langchain` library to do so : https://medium.com/@dmitri.mahayana/chatgpt-template-using-python-langchain-c201de474122